In [193]:
from dotenv import load_dotenv
import streamlit as st
import json
import pandas as pd
import redis
import os
import matplotlib.pyplot as plt
import plotly.express as px

In [194]:
st.set_page_config(page_title="What's in my Kitchen?", page_icon="🥘")
st.title("🥘 What's in my Kitchen?")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [195]:
load_dotenv()
r_host = os.environ.get('RD_HOST')
r_port = os.environ.get('RD_PORT')
r_pass = os.environ.get('RD_PASS')

print(os.getcwd())

In [196]:
with open('categories.txt') as c:
    data = c.read()
data = data.replace("'", "\"")
categories = json.loads(data)

In [197]:
@st.cache_data
def redis_call(host, port, password):

    r = redis.Redis(
        host=host,
        port=port,
        password=password)

    keys = r.keys()
    values = r.mget(keys)

    data = {}

    for key, value in zip(keys, values):
        data[f"{key.decode()}"] = f"{value.decode()}"

    return data

In [198]:
data = redis_call(r_host, r_port, r_pass)

In [199]:
df = pd.DataFrame()

for key, value in data.items():
    if key != 'key':
        temp_df = pd.DataFrame.from_dict(json.loads(value), orient='index').T
        temp_df.index = [key]
        df = pd.concat([df, temp_df])
    else:
        pass

st.dataframe(df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [200]:
df.head()

,quantity,unit
Chicken Breast,0,gram
Cooking Oil,496,mililitre
Sugar,-1,kilogram
Ginger Paste,498,gram
Olive Oil,498,mililitre


In [201]:
def categorize(categories, grocery_data):
    # create new column
    grocery_data['category'] = ''

    # categorize
    for index, _ in grocery_data.iterrows():
        item_name = index.lower()
        for category, keywords in categories.items():
            for keyword in keywords:
                if keyword in item_name:
                    grocery_data.at[index, 'category'] = category
                    break

    return grocery_data

In [202]:
categorize(categories, df).head()

,quantity,unit,category
Chicken Breast,0,gram,Meat
Cooking Oil,496,mililitre,Oil
Sugar,-1,kilogram,Bakery
Ginger Paste,498,gram,Spices
Olive Oil,498,mililitre,Oil


In [203]:
st.dataframe(categorize(categories, df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

df.columns

In [204]:
def sort_catagory(dataFrame):
    dataFrame = dataFrame.sort_values(by='category')
    return dataFrame

In [205]:
sort_catagory(df).head()

,quantity,unit,category
Sugar,-1,kilogram,Bakery
Durum,2,Packet,Bakery
Salt,250,gram,Bakery
Mushroom,4,Cans,Canned goods
Kidney Beans,-99,kilogram,Canned goods


In [206]:
grocery_list = pd.read_csv('D:\Codes\lynx_ai\pages\Grocery_List_I.csv') 
grocery_list.head()

,current items,quantity,unit
0,Rice,1,Kilogram
1,Eggs,10,Piece
2,Musli,2,Kilogram
3,Durum,8,piece
4,Chicken,1,Kilogram


In [207]:
def lower_case(s):
    if isinstance(s, str):
        return s.lower()
    else:
        return s
    
grocery_list = grocery_list.applymap(lower_case)

In [208]:
grocery_list

,current items,quantity,unit
0,rice,1,kilogram
1,eggs,10,piece
2,musli,2,kilogram
3,durum,8,piece
4,chicken,1,kilogram
5,tomato puree,250,gram
6,mushroom,250,gram
7,kidney beans,250,gram
8,sugar,1,kilogram
9,carrot,1,kilogram


In [209]:
def unit_conversion(dataFrame):
    conversion_factors = {
        'kilogram': {'gram': 1000},
        'litre': {'milliLitre': 1000},
        'ounce': {'gram': 28.3495},
        'pound': {'gram': 453.592},
        'teaspoon': {'milliliter': 4.92892},
        'tablespoon': {'milliliter': 14.7868},
        'cup': {'milliliter': 236.588},
    }

    for from_unit, to_units in conversion_factors.items():
        for to_unit, factor in to_units.items():
            mask = (dataFrame['unit'] == from_unit) & (dataFrame['unit'] != to_unit)
            dataFrame.loc[mask, 'quantity'] *= factor
            dataFrame.loc[mask, 'unit'] = to_unit
    return dataFrame

In [210]:
unit_conversion(grocery_list).head()

,current items,quantity,unit
0,rice,1000,gram
1,eggs,10,piece
2,musli,2000,gram
3,durum,8,piece
4,chicken,1000,gram


In [211]:
def to_sentence_case(dataFrame):
    for column in dataFrame.select_dtypes(include=['object']):
        dataFrame[column] = dataFrame[column].str.lower().apply(lambda x: x.capitalize())
    return dataFrame

In [212]:
to_sentence_case(grocery_list).head()

,current items,quantity,unit
0,Rice,1000,Gram
1,Eggs,10,Piece
2,Musli,2000,Gram
3,Durum,8,Piece
4,Chicken,1000,Gram


In [213]:
st.dataframe(unit_conversion(df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [214]:
fig = px.bar(grocery_list, x='unit', y='quantity')
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
import sys
print(sys.version)

def plotting(dataFrame, index):

    piece_df = dataFrame.loc[dataFrame['unit'] == 'Piece']
    gram_df = dataFrame.loc[dataFrame['unit'] == 'gram']
    milliLitre_df = dataFrame.loc[dataFrame['unit'] == 'MilliLitre']

    if not piece_df.empty:
        p = piece_df.plot(kind='bar', unit=index, quantity='quantitquantity')
        p.set_xlabel('Current Items')
        p.set_ylabel('quantity (Pieces)')

    if not gram_df.empty:
        g = gram_df.plot(kind='bar', unit=index, quantity='quantity')
        g.set_xlabel('Current Items')
        g.set_ylabel('quantitquantity (gram)')

    if not milliLitre_df.empty:
        l = milliLitre_df.plot(kind='bar', unit=index, quantity='quantity')
        l.set_xlabel('Current Items')
        l.set_ylabel('quantity (Millilitre)')

    plt.show()